In [5]:
# %pip install bs4

In [ ]:
import requests
from bs4 import BeautifulSoup
import json
import warnings
from tqdm import tqdm  # 진행 상황 표시용

# SSL 인증서 검증 관련 경고 메시지 억제
warnings.filterwarnings('ignore', message='Unverified HTTPS request')

# 크롤링할 인덱스 범위
START_INDEX = 87128
END_INDEX = 87890

BASE_URL = "https://accident.knia.or.kr/define-content?index="
results = []

for idx in tqdm(range(START_INDEX, END_INDEX + 1), desc="크롤링 진행 중"):
    url = f"{BASE_URL}{idx}"
    
    try:
        # SSL 인증서 검증 무시 (보안 경고: 실제 서비스에서는 인증서 검증 권장)
        response = requests.get(url, verify=False, timeout=10)
        
        if (response.status_code == 200):
            bs = BeautifulSoup(response.text, 'html.parser')
            term_div = bs.find('div', class_='bbsview_title')
            
            if term_div:
                term = term_div.get_text(strip=True)
                desc_divs = bs.find_all('div', class_='bbsview_cate')
                desc_list = []
                
                # p tag 리스트 내용 추출
                for desc_div in desc_divs:
                    p_tags = desc_div.find_all('p')

                    for p in p_tags:
                        text = p.get_text(strip=True)

                        if text:
                            desc_list.append(text)
                
                # 용어와 설명이 모두 있는 경우만 추가
                if desc_list:
                    results.append({'term': term, 'desc': desc_list})

    except Exception as e:
        print(f"인덱스 {idx} 처리 중 오류 발생: {str(e)}")

print(f"\n{len(results)}개 용어 수집 >> 크롤링 완료")



# 파일 저장
output_filename = 'terms.json'
with open(output_filename, 'w', encoding='utf-8') as f:
    json.dump(results, f, ensure_ascii=False, indent=2)
    
print("파일 저장 완료")

크롤링 진행 중: 100%|██████████| 763/763 [02:45<00:00,  4.61it/s]


60개 용어 수집 >> 크롤링 완료
파일 저장 완료
